# Comparison of Classifiers for the Breast Cancer Prediction

In this assignment we have been given the task to create a Decision Tree Classifier and a Naive Bayes Classifier, and compare the results of the two, from the given dataset.

### *The dataset*
We have been given the dataset Breast Cancer Prediction dataset. It contains data from cell nucleus in breast tissue, which is designed to assist diagnosing breastcancer. Here is an overview of the given dataset.

 - Consists of 569 instances/observations. 
 - Has 30 numerical features. Excluding ID and diagnosis.
 - The features are real, meaning they will consist of floating point numbers.
 - Classification problem, where the goal is to classify cancer tumours, based on cell properties.


#### *What is the purpose of this dataset?*
The purpose of this dataset is as mentioned before, using data from cell nucleus in breast tissue to assist diagnosing breastcancer.

#### *What are the features?*
The features are numeric values that describe different properties of the tumor. Such as texture, radius, smoothness and etc.

#### *What are the targets?*
M and B, will act as our boolean outputs/targets. Where B is benign, which will act as our boolean 0. The M is malignant, the opposite of the benign, will act as our boolean 1.

In other words, it can be simplified to as follows. B is for when the tumor is benign, and M is tumor is malignant.


In [82]:
import pandas as pd


In [83]:
dataset = pd.read_csv('wdbc.data', sep=",", header=None)
#dataset.drop(columns=[0],axis=1)
dataset.head(50)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.147100,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.71190,0.26540,0.4601,0.11890
1,842517,M,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.070170,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.24160,0.18600,0.2750,0.08902
2,84300903,M,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.127900,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.45040,0.24300,0.3613,0.08758
3,84348301,M,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.105200,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.68690,0.25750,0.6638,0.17300
4,84358402,M,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.104300,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.40000,0.16250,0.2364,0.07678
5,843786,M,12.450,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.080890,...,15.470,23.75,103.40,741.6,0.17910,0.52490,0.53550,0.17410,0.3985,0.12440
6,844359,M,18.250,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.074000,...,22.880,27.66,153.20,1606.0,0.14420,0.25760,0.37840,0.19320,0.3063,0.08368
7,84458202,M,13.710,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.059850,...,17.060,28.14,110.60,897.0,0.16540,0.36820,0.26780,0.15560,0.3196,0.11510
8,844981,M,13.000,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.093530,...,15.490,30.73,106.20,739.3,0.17030,0.54010,0.53900,0.20600,0.4378,0.10720
9,84501001,M,12.460,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.085430,...,15.090,40.68,97.65,711.4,0.18530,1.05800,1.10500,0.22100,0.4366,0.20750


## Pre-prossesing of data
Our pre-prossesing will consist of removing the ID, since it's unusable.
Also, since this is a .data file and not a .csv file, the first row is now a "header" for the other coloums, which it shouldn't be.